In [44]:
def daterange_by_interval(start_date, end_date, interval=2):
    """
    Create a list of intervals for sql between usage
    :param start_date: First date in the range, inclusive.
    :param end_date: Last date in the range, inclusive.
    :param interval: integer representing days
    :return: list of [start_date, end_date]
    """
    if not interval:
        raise Exception('Invalid interval - Should be greater than zero')
        
    cur_date = start_date
    
    while cur_date <= end_date:
        target_date = cur_date + datetime.timedelta(days=interval-1)
        if target_date == end_date:
            target_date = end_date + datetime.timedelta(days=1)
        yield (cur_date, target_date)
        cur_date = target_date # + datetime.timedelta(days=1) # offset for next chunk

In [45]:
import datetime

start_date = datetime.datetime.strptime('01-01-2009', '%m-%d-%Y').date()
end_date = datetime.datetime.strptime('06-30-2015', '%m-%d-%Y').date()

dates = [date for date in daterange_by_interval(start_date, end_date, interval=2)]
print((dates[0], dates[1], dates[-1]))

((datetime.date(2009, 1, 1), datetime.date(2009, 1, 2)), (datetime.date(2009, 1, 2), datetime.date(2009, 1, 3)), (datetime.date(2015, 6, 29), datetime.date(2015, 7, 1)))


In [4]:
dates[0][0].isoformat()

'2009-01-01'

In [38]:
query = """SELECT COUNT(1) FROM nyc_taxi_fare_prediction.test_features WHERE key_temp BETWEEN '{start_date}' AND '{end_date}'"""

In [39]:
import teradatasql

values = list()

with teradatasql.connect(host='sdt25548.labs.teradata.com', user='dbc', password='dbc') as conn:
    with conn.cursor() as cursor:
        for start_date, end_date in dates:
            cursor.execute(query.format(start_date=start_date.isoformat(),
                                        end_date=end_date.isoformat()))
            rows = cursor.fetchall()
            row_count = rows[0][0]
            values.append(row_count)

In [40]:
sum(values)

10744136